In [ ]:
!pip install datasets

squad数据集加载

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering
from torch.utils.data import DataLoader, TensorDataset
import torch


from datasets import load_dataset

# 下载并加载SQuAD数据集
squad_dataset = load_dataset('squad')

train_squad = squad_dataset['train']
test_squad = squad_dataset['validation']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

这里为了快速看到效果，只切片了一部分数据

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer中标点符号算token，但是空格不算
train_encoded = tokenizer(text=train_squad['context'][:1000],text_pair=train_squad['question'][:1000],truncation=True, return_tensors='pt',max_length=512,pad_to_max_length=True)
test_encoded = tokenizer(text = test_squad['context'][:100],text_pair = test_squad['question'][:100],truncation=True, return_tensors='pt',max_length=512,pad_to_max_length=True)



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


由于数据集提供的格式并不是bert需要的，所以我们要做一些预处理，注意start和end是指token的位置（标点符号也算token）

In [ ]:
import string
train_temp = train_squad[:1000]
test_temp = test_squad[:100]

def count_words(text):
    # 使用split()函数将文本分割成单词列表
    words = text.split()
    # 返回单词的数量
    return len(words)

def count_words_and_punctuations(idx,text):
    # 截取到指定索引位置的文本
    sub_text = text[:idx]

    # 使用split()函数将文本分割成单词列表
    words = sub_text.split()

    # 计算单词数量
    word_count = len(words)

    # 计算标点符号数量
    punctuation_count = sum(1 for char in sub_text if char in string.punctuation)

    return word_count+punctuation_count

train_start = [count_words_and_punctuations(train_temp['answers'][i]['answer_start'][0],train_temp['context'][i])  for i in range(1000)]
train_end = [count_words_and_punctuations(train_temp['answers'][i]['answer_start'][0] + count_words(train_temp['answers'][i]['text'][0]),train_temp['context'][i]) for i in range(1000)]

test_start = [count_words_and_punctuations(test_temp['answers'][i]['answer_start'][0],test_temp['context'][i])  for i in range(100)]
test_end = [count_words_and_punctuations(test_temp['answers'][i]['answer_start'][0] + count_words(test_temp['answers'][i]['text'][0]),test_temp['context'][i]) for i in range(100)]

In [ ]:
train_data = TensorDataset(train_encoded['input_ids'],train_encoded['attention_mask'],train_encoded['token_type_ids'],torch.tensor(train_start),torch.tensor(train_end))
test_data = TensorDataset(test_encoded['input_ids'],test_encoded['attention_mask'],test_encoded['token_type_ids'],torch.tensor(test_start),torch.tensor(test_end))
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=True)

train model

In [ ]:
import torch.nn as nn
from transformers import BertForQuestionAnswering
from torch.nn.utils import clip_grad_norm_
import math


model = BertForQuestionAnswering.from_pretrained('bert-base-uncased').cuda()
optim = torch.optim.Adam(model.parameters(), lr=0.0001)
for ep in range(10):
    total_loss = 0
    for b in train_dataloader:
        iids, am, tids, s, e = b
        out = model(input_ids=iids.cuda(), attention_mask=am.cuda(), token_type_ids=tids.cuda(), start_positions=s.cuda(),end_positions = e.cuda())
        optim.zero_grad()
        out.loss.backward()
        clip_grad_norm_(model.parameters(), max_norm=1.0)  # 梯度裁剪

        optim.step()
        total_loss += out.loss.item()
    print(f'Epoch {ep+1}/{10}, Loss: {total_loss}')




Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10, Loss: 610.0084509849548
Epoch 2/10, Loss: 454.3976652622223
Epoch 3/10, Loss: 295.47154557704926
Epoch 4/10, Loss: 184.32552227377892
Epoch 5/10, Loss: 136.62112797051668
Epoch 6/10, Loss: 117.89112965762615
Epoch 7/10, Loss: 85.77475959435105
Epoch 8/10, Loss: 68.77082483749837
Epoch 9/10, Loss: 63.80306768929586
Epoch 10/10, Loss: 54.0668830415234


test model

In [ ]:
# 加载tokenizer

# 准备问题和上下文
model.cpu()
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
# tokenize问题和上下文
encoded_dict = tokenizer.encode_plus(
    question, text, return_tensors='pt')

with torch.no_grad():  # 确保不计算梯度
    outputs = model(**encoded_dict)
# 预测答案的起始和结束位置
start_index = outputs.start_logits.argmax()
end_index = outputs.end_logits.argmax()
# 使用tokenizer解码以获取原始文本中的答
predict_answer_tokens = encoded_dict.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'puppet [SEP]'